# Exercice - Machine Learning pour l'analysis de sentiment

Dans ce notebook, nous allons entraîner et évaluer un classifieur afin de prédire le sentiment de critiques de films sur la plateforme iMDb. C'est un cas d'école classique permettant d'aborder les thématique de la _classification binaire_ et du _traitement automatique des langues_ (_NLP_) de façon ludique.

Jeu de données :
- Source : https://ai.stanford.edu/~amaas/data/sentiment/
- CSV : https://github.com/Ankit152/IMDB-sentiment-analysis/raw/master/IMDB-Dataset.csv

In [ ]:
import pandas

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

## Préparation du jeu de données

Préparons notre jeu de données :

1. Extraction
2. Séparation d'un jeu d'entraînement et d'un jeu de test
3. Analyse rapide

In [ ]:
dataset_url = "https://github.com/Ankit152/IMDB-sentiment-analysis/raw/master/IMDB-Dataset.csv"
dataset = pandas.read_csv(dataset_url)
train_dataset, test_dataset = train_test_split(dataset, random_state=42)

In [ ]:
train_dataset.head()

In [ ]:
train_dataset.info()

## Extraction de caractéristiques

Les algorithmes de machine learning ne fonctionnent qu'avec des valeurs numériques. Or lorsque notre problème considéré relève du NLP, nous n'avons à notre disposition uniquement des chaînes de caractères. Afin de pouvoir extraire des _caractéristiques_ (_features_) numériques, voici la marche à suivre :

1. _Tokenisation_ : transformer les phrases en _tokens_ / _symboles_ (par exemple en découpant par mots)
2. _Vectorisation_ : transformer les _tokens_ en valeurs numériques (par exemple avec un _bag-of-words_ pour compter la fréquence de chaque _token_)

La bibliothèque `scikit-learn` fournit le vectoriseur `CountVectorizer` permettant de tokeniser puis d'appliquer une vectorisation de type _bag-of-words_ (compte la fréquence des mots et ne garde que les N plus fréquents) :

In [ ]:
bow = CountVectorizer(analyzer="word", max_features=100)
bow.fit(train_dataset["review"])

On peut observer le dictionnaire construit par le vectoriseur, avec pour chaque token identifié la fréquence associée :

In [ ]:
bow.vocabulary_

Appliquons la transformation à notre jeu de données :

In [ ]:
X_train = bow.transform(train_dataset["review"]).toarray()
X_train

## Entrainement d'un modèle

Notre jeu de données d'entrainement est prêt, nous pouvons passer à l'entrainement d'un modèle.

In [ ]:
y_train = train_dataset["sentiment"]
lr = LogisticRegression(max_iter=10000)
lr.fit(X_train, y_train)

## Evaluation d'un modèle

Une fois l'entraînement d'un modèle effectué, il convient d'évaluer ses performance. Notre problème est de type classification binaire, nous pouvons utiliser les outils d'évaluation suivants (liste non-exhaustive) :
- Visualisation : matrice de confusion
- Métriques : justesse (_accuracy_), f1-score, précision, rappel (_recall_)

Commençons par préparer le jeu de test **avec strictement les même traitements que le jeu d'entraînement** puis effectuer les prédictions sur le jeu de test :

In [ ]:
X_test = bow.transform(test_dataset["review"])
y_test = test_dataset["sentiment"]
y_pred = lr.predict(X_test)

Affichons la matrice de confusion du modèle :

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=lr.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=lr.classes_)
_ = disp.plot()

Enfin, nous pouvons décortiquer la matrice de confusion avec les métriques de classification :

In [ ]:
print(classification_report(y_test, y_pred))

## Effectuer des prédictions

Nous pouvons utiliser le modèle entraîné pour effectuer des prédictions :

In [ ]:
review = "Had a great time"
processed_review = bow.transform([review])
lr.predict(processed_review)

In [ ]:
review = "I hate this movie"
processed_review = bow.transform([review])
lr.predict(processed_review)

Nous pouvons également observer les limites du modèle actuel :

In [ ]:
review = "I love this movie"
processed_review = bow.transform([review])
lr.predict(processed_review)

Voici un petit programme interactif permettant de générer des prédictions de sentiments dynamiques :

In [ ]:
review = input("Review: ")
processed_review = bow.transform([review])
predictions = lr.predict(processed_review)
print(predictions[0])

## Question

Le modèle actuel fonctionne mais avec une performance modérée : il est meilleur qu'un aléatoire mais il peut facilement se tromper dans ses prédictions :
- D'après-vous, le modèle est-il en _sous-apprentissage_ (_underfitting_) ou _sur-apprentissage_ (_overfitting_) ?
- Pourquoi ?

## Pour aller plus loin

Si vous souhaitez aller plus loin, voici des pistes d'amélioration :

- (facile) Augmenter la taille du dictionnaire du vectoriseur _bag-of-words_
- (facile) Changer le vectoriseur par un TF-IDF
- (intermédiaire) Changer le modèle par un SVM / un arbre de décision / une forêt aléatoire et comparer les résultats
- (intermédiaire) Analyser et nettoyer le jeu de données source afin de ne garder uniquement les termes pertinents et ne pas fausser les modèles
- (avancé) Effectuer une recherche d'hyper-paramètres afin d'optimiser au mieux un modèle
- (avancé) Refactorer le code en utilisant la notion de _pipeline_ afin de ne pas avoir à ré-effectuer les traitement pour chaque jeu de données